In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import DateType
from pyspark.sql.types import DecimalType, IntegerType

catalogo = "medalhao"
gold_db_name = "gold"


In [0]:
%sql
USE CATALOG medalhao;

CREATE SCHEMA IF NOT EXISTS Gold;


In [0]:
ft_vendas_consumidor_local = (
    pedidos_silver_df
    .join(consumidores_silver_df, "id_consumidor", "inner")
    .select(
        F.col("id_pedido"),
        F.col("id_consumidor"),
        F.col("valor_total_pago_brl").cast("decimal(12,2)").alias("valor_total_pedido_brl"),
        F.col("cidade"),
        F.col("estado"),
        F.to_date("data_pedido").alias("data_pedido")
    )
)

ft_vendas_consumidor_local.write.format("delta").mode("overwrite").saveAsTable(
    f"{catalogo}.{gold_db_name}.ft_vendas_consumidor_local"
)
ft_vendas_consumidor_local.display()

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_total_compras_por_consumidor AS
SELECT 
    cidade,
    estado,
    COUNT(DISTINCT id_pedido) AS quantidade_vendas,
    ROUND(SUM(valor_total_pedido_brl), 2) AS valor_total_localidade
FROM medalhao.gold.ft_vendas_consumidor_local
GROUP BY cidade, estado
ORDER BY valor_total_localidade DESC;
SELECT *
FROM medalhao.gold.view_total_compras_por_consumidor


In [0]:
pedidos_df = spark.table(f"medalhao.silver.pedidos_silver")
itens_pedidos_df = spark.table(f"medalhao.silver.itens_pedidos_silver")
consumidores_df = spark.table(f"medalhao.silver.consumidores_silver")

itens_vendedor_agg = itens_pedidos_df.select("id_pedido", "id_vendedor").distinct()

ft_atrasos_pedidos_local_vendedor_df = (
    pedidos_df.alias("p")
    .join(itens_vendedor_agg.alias("i"), 
          on="id_pedido",
          how="inner")
    .join(consumidores_df.alias("c"), 
          on="id_consumidor", 
          how="inner")
    .select(
        F.col("p.id_pedido").alias("id_pedido"),
        F.col("i.id_vendedor").alias("id_vendedor"),
        F.col("p.id_consumidor").alias("id_consumidor"),
        F.col("p.entrega_no_prazo").alias("entrega_no_prazo"),
        F.col("p.tempo_entrega_dias").alias("tempo_entrega_dias"),
        F.col("p.tempo_entrega_estimado_dias").alias("tempo_entrega_estimado_dias"),
        F.col("c.cidade").alias("cidade"),
        F.col("c.estado").alias("estado")
    )
)

ft_atrasos_pedidos_local_vendedor_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{gold_db_name}.ft_atrasos_pedidos_local_vendedor")


In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_tempo_medio_entrega_localidade AS
SELECT
    cidade,
    estado,
    CAST(ROUND(AVG(tempo_entrega_dias), 2) AS DECIMAL(12,2)) AS tempo_medio_entrega,
    CAST(ROUND(AVG(tempo_entrega_estimado_dias), 2) AS DECIMAL(12,2)) AS tempo_medio_estimado,
    CASE
        WHEN AVG(tempo_entrega_dias) > AVG(tempo_entrega_estimado_dias) THEN 'SIM'
        ELSE 'NÃO'
    END AS entrega_maior_que_estimado
FROM
    medalhao.gold.ft_atrasos_pedidos_local_vendedor
GROUP BY
    cidade,
    estado
;
select * from medalhao.gold.view_tempo_medio_entrega_localidade

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendedor_pontualidade AS
SELECT
    id_vendedor,
    COUNT(DISTINCT id_pedido) AS total_pedidos,
    COUNT(DISTINCT CASE WHEN entrega_no_prazo = 'Não' THEN id_pedido END) AS total_atrasados,
    CAST(
        ROUND(
            (COUNT(DISTINCT CASE WHEN entrega_no_prazo = 'Não' THEN id_pedido END) * 100.0) / COUNT(DISTINCT id_pedido),
            2
        ) AS DECIMAL(12,2)
    ) AS percentual_atraso
FROM
    medalhao.gold.ft_atrasos_pedidos_local_vendedor
GROUP BY
    id_vendedor
;
select * from medalhao.gold.view_vendedor_pontualidade

In [0]:
pedidos_df = spark.table("medalhao.silver.pedidos_silver")
data_range = pedidos_df.agg(
    F.min("pedido_compra_timestamp").alias("min_data"),
    F.max("pedido_compra_timestamp").alias("max_data")
).collect()[0]

data_min = data_range["min_data"]
data_max = data_range["max_data"]

date_df = spark.createDataFrame([(data_min, data_max)], ["start_date", "end_date"])

dm_tempo_df = (
    date_df.withColumn(
        "date_sequence", 
        F.expr(f"sequence(to_date('{data_min}'), to_date('{data_max}'), interval 1 day)")
    )
    .withColumn("sk_tempo", F.explode("date_sequence"))
    .select(
        F.col("sk_tempo").cast(DateType()).alias("sk_tempo"),
        F.year("sk_tempo").alias("ano"),
        F.quarter("sk_tempo").alias("trimestre"),
        F.month("sk_tempo").alias("mes"),
        F.weekofyear("sk_tempo").alias("semana_do_ano"),
        F.dayofmonth("sk_tempo").alias("dia"),
        F.dayofweek("sk_tempo").alias("dia_da_semana_num"),
        F.date_format("sk_tempo", "EEEE").alias("dia_da_semana_nome"),
        F.date_format("sk_tempo", "MMMM").alias("mes_nome"),
        F.when(F.dayofweek("sk_tempo").isin(1, 7), F.lit("Sim")).otherwise(F.lit("Não")).alias("eh_fim_de_semana")
    )
)

dm_tempo_df = dm_tempo_df.withColumn(
    "dia_da_semana_nome",
    F.when(F.col("dia_da_semana_nome") == "Sunday", "Domingo")
    .when(F.col("dia_da_semana_nome") == "Monday", "Segunda-feira")
    .when(F.col("dia_da_semana_nome") == "Tuesday", "Terça-feira")
    .when(F.col("dia_da_semana_nome") == "Wednesday", "Quarta-feira")
    .when(F.col("dia_da_semana_nome") == "Thursday", "Quinta-feira")
    .when(F.col("dia_da_semana_nome") == "Friday", "Sexta-feira")
    .when(F.col("dia_da_semana_nome") == "Saturday", "Sábado")
    .otherwise(F.col("dia_da_semana_nome"))
).withColumn(
    "mes_nome",
    F.when(F.col("mes_nome") == "January", "Janeiro")
    .when(F.col("mes_nome") == "February", "Fevereiro")
    .when(F.col("mes_nome") == "March", "Março")
    .when(F.col("mes_nome") == "April", "Abril")
    .when(F.col("mes_nome") == "May", "Maio")
    .when(F.col("mes_nome") == "June", "Junho")
    .when(F.col("mes_nome") == "July", "Julho")
    .when(F.col("mes_nome") == "August", "Agosto")
    .when(F.col("mes_nome") == "September", "Setembro")
    .when(F.col("mes_nome") == "October", "Outubro")
    .when(F.col("mes_nome") == "November", "Novembro")
    .when(F.col("mes_nome") == "December", "Dezembro")
    .otherwise(F.col("mes_nome"))
)

dm_tempo_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{gold_db_name}.dm_tempo")


In [0]:
itens_df = spark.table("medalhao.silver.itens_pedidos_silver")
pedidos_df = spark.table("medalhao.silver.pedidos_silver")
cotacao_df = spark.table("medalhao.silver.dm_cotacao_dolar")
avaliacoes_df = spark.table("medalhao.silver.avaliacoes_pedidos_silver")
produtos_df = spark.table("medalhao.silver.produtos_silver")

avaliacoes_agg = avaliacoes_df.groupBy("id_pedido").agg(
    F.round(F.avg("avaliacao"), 2).cast(DecimalType(3, 2)).alias("avaliacao_pedido")
)

base_df = (
    itens_df.alias("i")
    .join(pedidos_df.alias("p"), 
          on="id_pedido", 
          how="inner")
    .join(
        cotacao_df.alias("d"), 
        F.to_date(F.col("p.pedido_compra_timestamp")) == F.col("d.data"), 
        how="left"
    )
    .join(avaliacoes_agg.alias("a"), 
          on="id_pedido", 
          how="left")
    .select(
        F.col("i.id_pedido"),
        F.col("i.id_item"),
        F.col("p.id_consumidor").alias("fk_cliente"),
        F.col("i.id_produto").alias("fk_produto"),
        F.col("i.id_vendedor").alias("fk_vendedor"),
        F.to_date(F.col("p.pedido_compra_timestamp")).alias("fk_tempo"),
        F.col("p.status").alias("status_pedido"),
        F.col("p.tempo_entrega_dias").cast(IntegerType()).alias("tempo_entrega_dias"),
        F.col("p.entrega_no_prazo"),
        F.col("i.preco_BRL").cast(DecimalType(12, 2)).alias("valor_produto_brl"),
        F.col("i.preco_frete").cast(DecimalType(12, 2)).alias("valor_frete_brl"),
        (F.col("i.preco_BRL") + F.col("i.preco_frete")).cast(DecimalType(12, 2)).alias("valor_total_item_brl"),
        F.col("d.cotacao_dolar").cast(DecimalType(8, 4)).alias("cotacao_dolar"),
        F.col("a.avaliacao_pedido")
    )
)

In [0]:
ft_vendas_geral_df = base_df.withColumn(
    "valor_produto_usd",
    F.round(F.col("valor_produto_brl") / F.col("cotacao_dolar"), 2).cast(DecimalType(12, 2))
).withColumn(
    "valor_frete_usd",
    F.round(F.col("valor_frete_brl") / F.col("cotacao_dolar"), 2).cast(DecimalType(12, 2))
).withColumn(
    "valor_total_item_usd",
    F.round(F.col("valor_total_item_brl") / F.col("cotacao_dolar"), 2).cast(DecimalType(12, 2))
).fillna(0, subset=["valor_produto_usd", "valor_frete_usd", "valor_total_item_usd"])


ft_vendas_geral_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{gold_db_name}.ft_vendas_geral")
print("✅ Tabela gold.ft_vendas_geral criada com sucesso!")

display(spark.table(f"{catalogo}.{gold_db_name}.ft_vendas_geral"))

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendas_por_periodo AS
SELECT
    t.ano,
    t.trimestre,
    t.mes,
    t.mes_nome,
    t.dia,
    t.dia_da_semana_num,
    COUNT(DISTINCT v.id_pedido) AS total_pedidos,
    COUNT(v.id_pedido) AS total_itens,
    SUM(v.valor_total_pedido_brl) AS receita_total_brl,
    CAST(ROUND(SUM(v.valor_total_pedido_brl) / COUNT(v.id_pedido), 2) AS DECIMAL(12,2)) AS ticket_medio_brl
FROM
    medalhao.gold.ft_vendas_consumidor_local v
INNER JOIN
    medalhao.gold.dm_tempo t ON DATE(v.data_pedido) = t.sk_tempo
GROUP BY
    t.ano, t.trimestre, t.mes, t.mes_nome, t.dia, t.dia_da_semana_num
ORDER BY
    t.ano, t.mes, t.dia
;
select * from medalhao.gold.view_vendas_por_periodo

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_top_produto AS
SELECT
    v.fk_produto AS id_produto,
    COALESCE(cat.nome_produto_en, p.categoria_produto) AS categoria_produto,
    COUNT(v.id_item) AS quantidade_vendida,
    COUNT(DISTINCT v.id_pedido) AS total_pedidos,
    SUM(v.valor_total_item_brl) AS receita_brl,
    SUM(v.valor_total_item_usd) AS receita_usd,
    CAST(ROUND(AVG(v.valor_produto_brl), 2) AS DECIMAL(12, 2)) AS preco_medio_brl,
    CAST(ROUND(AVG(v.avaliacao_pedido), 2) AS DECIMAL(3, 2)) AS avaliacao_media,
    CAST(ROUND(AVG(p.peso_produto_gramas), 2) AS DECIMAL(8, 2)) AS peso_medio_gramas
FROM
    medalhao.gold.ft_vendas_geral v
INNER JOIN
    medalhao.silver.produtos_silver p ON v.fk_produto = p.id_produto
LEFT JOIN
    medalhao.silver.categoria_produtos_silver cat ON p.categoria_produto = cat.nome_produto_pt
GROUP BY
    v.fk_produto,
    COALESCE(cat.nome_produto_en, p.categoria_produto)
ORDER BY
    receita_brl DESC
;
select * from medalhao.gold.view_top_produto

In [0]:
%sql
CREATE OR REPLACE VIEW medalhao.gold.view_vendas_produtos_esteticos AS
WITH FashionSales AS (
    SELECT
        t.ano,
        t.mes,
        COALESCE(cat.nome_produto_en, p.categoria_produto) AS categoria_produto,
        v.id_pedido,
        v.id_item,
        v.valor_total_item_brl,
        v.valor_total_item_usd,
        v.avaliacao_pedido
    FROM
        medalhao.gold.ft_vendas_geral v
    INNER JOIN
        medalhao.gold.dm_tempo t ON v.fk_tempo = t.sk_tempo
    INNER JOIN
        medalhao.silver.produtos_silver p ON v.fk_produto = p.id_produto
    LEFT JOIN
        medalhao.silver.categoria_produtos_silver cat ON p.categoria_produto = cat.nome_produto_pt
    WHERE
        COALESCE(cat.nome_produto_en, p.categoria_produto) LIKE 'fashion%' 
)
SELECT
    ano,
    mes,
    categoria_produto,
    COUNT(DISTINCT id_pedido) AS total_pedidos,
    COUNT(id_item) AS total_itens_vendidos,
    SUM(valor_total_item_brl) AS receita_total_brl,
    SUM(valor_total_item_usd) AS receita_total_usd,
    CAST(ROUND(SUM(valor_total_item_brl) / COUNT(id_item), 2) AS DECIMAL(12, 2)) AS ticket_medio_brl,
    CAST(ROUND(SUM(valor_total_item_usd) / COUNT(id_item), 2) AS DECIMAL(12, 2)) AS ticket_medio_usd,
    CAST(ROUND(AVG(avaliacao_pedido), 2) AS DECIMAL(3, 2)) AS avaliacao_media
FROM
    FashionSales
GROUP BY
    ano,
    mes,
    categoria_produto
ORDER BY
    ano, mes, receita_total_brl DESC
;
select * from medalhao.gold.view_vendas_produtos_esteticos